# Face Detection and Recognition
This notebook provides a quick resource for exploring OpenCV's built-in face detection and recognition algorithms, based on the third section of the following youtube video: https://www.youtube.com/watch?v=oXlwWbU8l2o

Images of 'famous' people are used as examples, which were downloaded from the Labelled Faces in the Wild (LFW) dataset, available as part of the following Kaggle competition: https://www.kaggle.com/jessicali9530/lfw-dataset

The goal of Face Detection is to identify a rectangular bounding box around each face in an input image. Additional features may be detected as well, e.g. eyes, smile, etc. The goal of Face Recognition is to correctly identify the person based on their facial features. Common practice is to first apply face detection, then apply a facial recognition algorithm to identify the name of the person from features in the detected face image.

## References
- https://github.com/Kaggle/kaggle-api (you can use the api to download the data)
- https://pythonprogramming.net/haar-cascade-object-detection-python-opencv-tutorial/
- https://towardsdatascience.com/a-dog-detector-and-breed-classifier-4feb99e1f852 (also see https://github.com/HenryDashwood/dog_breed_classifier)
- https://keras.io/guides/transfer_learning/
- https://towardsdatascience.com/face-detection-in-2-minutes-using-opencv-python-90f89d7c0f81
- https://www.pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
- https://stackoverflow.com/questions/20801015/recommended-values-for-opencv-detectmultiscale-parameters
- https://docs.opencv.org/4.1.0/d5/d54/group__objdetect.html
- 

In [1]:
import cv2 as cv
import numpy as np
import os
import random

In [2]:
dir_root = r"/home/fdpearce/Documents/Projects/data/Images/LFW/lfw-deepfunneled/lfw-deepfunneled"
# Name must match folder name exactly after a single modification: spaces, " ", are converted to underscores, "_"
people = ["George W Bush", "Laura Bush", "Vladimir Putin", "Gloria Macapagal Arroyo", "Arnold Schwarzenegger", "Megawati Sukarnoputri", \
          "Hugo Chavez", "Serena Williams", "Colin Powell", "Junichiro Koizumi", "Jennifer Capriati"]

In [3]:
num_people_with_most_img = 25
people_folder = os.listdir(dir_root)
num_images_per_folder = [len(os.listdir(os.path.join(dir_root, p))) for p in people_folder]
# Get list of tuples containing (Person Name, # of Images), sorted by # of Images in descending order
sort_people_num_img = sorted(zip(people_folder, num_images_per_folder), reverse=True, key=lambda pair: pair[1])
print(f"# of People with Images Available = {len(people_folder)}")
print(f"Top {num_people_with_most_img} People With Most Images, # of Images")
print(*sort_people_num_img[:num_people_with_most_img], sep="\n")

# of People with Images Available = 5749
Top 25 People With Most Images, # of Images
('George_W_Bush', 530)
('Colin_Powell', 236)
('Tony_Blair', 144)
('Donald_Rumsfeld', 121)
('Gerhard_Schroeder', 109)
('Ariel_Sharon', 77)
('Hugo_Chavez', 71)
('Junichiro_Koizumi', 60)
('Jean_Chretien', 55)
('John_Ashcroft', 53)
('Serena_Williams', 52)
('Jacques_Chirac', 52)
('Vladimir_Putin', 49)
('Luiz_Inacio_Lula_da_Silva', 48)
('Gloria_Macapagal_Arroyo', 44)
('Arnold_Schwarzenegger', 42)
('Jennifer_Capriati', 42)
('Laura_Bush', 41)
('Lleyton_Hewitt', 41)
('Hans_Blix', 39)
('Alejandro_Toledo', 39)
('Nestor_Kirchner', 37)
('Andre_Agassi', 36)
('Alvaro_Uribe', 35)
('Megawati_Sukarnoputri', 33)


In [4]:
# Functions for object detection, facial recognition, and visualizing results
def create_training_data(dir_root, people, person_img_max, haar_path, haar_params, detect_objects=False, random_seed=0, verbose=False):
    """Create training data from the following inputs:
    dir_root = a string containing base directory with subfolders containing images for each person
    people = a list containing string values, with each string containing a person's name. These are the folder names, EXCEPT for spaces instead of underscores
    person_img_max = an integer specifying the maximum number of training images to return for each person. Set this to a value larger than that
                     maximum # of images available if you want to process all images
    haar_path = a string containing the path to the haar cascade class definition xml file. Files for different features (e.g. face, eyes, etc) can
                be downloaded from the following link: https://github.com/opencv/opencv/blob/master/data/haarcascades/
    haar_params = a dictionary containing the parameters to pass to the detectMultiScale method of the haar cascade class, e.g. scaleFactor, minNeighbors
    detect_objects = a boolean-like value (True/False, 1/0, etc) that turns on/off object detection. Set to false to conduct a dry run that checks
                     how many image files will be processed for each person
    random_seed = an integer that determines the order of the random shuffle applied to images before selecting the training samples
    verbose = a boolean-like value that, when truthy, prints additional details during execution for validation/debugging purposes
    Output is a tuple with two values:
    features = a list containing zero or more lists, with each list containing four values (x, y, w, h) that define the rectangle containing the detected object
    labels = a list containing zero or more integer values, with each int specifying the index to a specific person in the input list of people used for training
    """
    random.seed(random_seed)
    features = []
    labels = []
    for person in people:
        label = people.index(person)
        path = os.path.join(dir_root, "_".join(person.split(" ")))
        img_files = os.listdir(path)
        random.shuffle(img_files)
        img_num = 0
        total_detected = 0
        for img in img_files[:min(person_img_max, len(img_files))]:
            img_path = os.path.join(path, img)
            img_num += 1
            print(f"Working on Image # {img_num}: {img}")
            if detect_objects:
                detected_feature, detected_label = detect_primary_objects(img_path, haar_path, label, verbose=verbose, **haar_params)
                if detected_label >= 0:
                    total_detected += 1
                    features.append(detected_feature)
                    labels.append(detected_label)
        print(f"{img_num} training images with {total_detected} objects identified for {person}")
    return features, labels

def detect_primary_objects(gray, haar_path, max_iter=10, boost_flag=True, verbose=False, **haar_params):
    """Identify the "primary", or least likely to be a false positive, object detected within the input grayscale image, gray.
    The type of object detected by haar_cascade is determined by the haar cascade class xml file that was provided in 
    the haar_path parameter input to create_training_data.
    """
    haar_cascade = cv.CascadeClassifier(haar_path)
    detected_objects = haar_cascade.detectMultiScale(gray, **haar_params)
    num_detected = len(detected_objects)
    num_detected_prev = num_detected
    if verbose:
        print(f"Initial # of Objects Detected = {num_detected}")
    num_iter = 0
    boost_factor = 1
    while num_detected != 1 and num_iter != max_iter:
        num_iter += 1
        if verbose:
            print(f"Iteration # = {num_iter}")
        # Update minNeighbors value in copy of params dict
        if num_iter == 1:
            haar_params_new = haar_params.copy()
        elif num_iter == max_iter:
            print(f"Maximum # of iterations ({max_iter}) reached!")
        if num_detected < 1 and haar_params_new['minNeighbors'] > 1:
            haar_params_new['minNeighbors'] -= boost_factor
        elif num_detected > 1:
            haar_params_new['minNeighbors'] += 2 * boost_factor
        else:
            print("Unable to detect object in input image")
            print(f"Verify that either 1) num_detected is zero ({num_detected==0}) and minNeighbors is one ({haar_params_new['minNeighbors']==1})")
            print(f"OR 2) the maximum # of iterations has been reached ({num_iter==max_iter})")
            print("If either of these scenarios occurs, consider changing the input scaleFactor and/or initial minNeighbors value. If neither 1) or 2) applies, then there is an unknown bug somewhere that should be investigated!!!")
        if verbose:
            print(f"minNeighbors = {haar_params_new['minNeighbors']}")
        detected_objects = haar_cascade.detectMultiScale(gray, **haar_params_new)
        num_detected = len(detected_objects)
        if num_detected == num_detected_prev and boost_flag:
            boost_factor += 1
        else:
            boost_factor = 1
        num_detected_prev = num_detected
    if verbose:
        print(f"Final # of Objects Detected = {num_detected}")
    return detected_objects

def detect_image_objects(img_path, haar_path, label=-1, verbose=False, **haar_params):
    """Detect object(s) in the image located at img_path, using the haar object defined in
    the xml file located at haar_path where
    img_path = a string defining the path to the image file for object detection
    haar_path = a string defining the path to the haar cascade class definition xml file as described in create_training_data
    label = an integer specifying the index to a specific person in the people list that is the primary person in the image at img_path
            When the default value of -1 is provided, then no label is returned, essentially the default is a non-training mode
    verbose = a boolean-like value that, when truthy, prints additional details during execution for validation/debugging purposes
    haar_params = a dictionary containing the parameters to pass to the detectMultiScale method of the haar cascade class
                  Valid values include scaleFactor (default=1.1), minNeighbors (default=3), and minSize. 
    """
    img_array = cv.imread(img_path)
    gray = cv.cvtColor(img_array, cv.COLOR_BGR2GRAY)
    # Detect "primary" object in image
    detected_rects = detect_primary_objects(gray, haar_path, verbose=verbose, **haar_params)
    # detected_rects returns a list that should only have one value, so only one is considered below
    # Exception will alert to any issues, and skip detected object so it is NOT included in features and labels
    try:
        (x, y, w, h) = detected_rects[0]
    except Exception as e:
        print(f"The following error occurred when performing object detection for the image at {img_path}:")
        print(e)
        x = None
    if verbose:
        print(*detected_rects[0], sep=", ")
    if isinstance(x, int):
        obj_roi = gray[y:y+h, x:x+w]
        obj_label = label
    else:
        obj_roi = None
        obj_label = -1
    if obj_label > -1:
        return obj_roi, obj_label
    else:
        return obj_roi

def show_frame(frame, frame_title):
    cv.imshow(frame_title, frame)
    cv.waitKey(0)
    cv.destroyAllWindows()
    
def show_detected_objects(detected_frame, detected_rect, frame_to_show=None, print_detected=False, rect_color=(255, 255, 255), rect_thickness=2):
    """Display source image with detected object(s) outlined, and optionally display an image focused around each detected object based on a
    different input image, frame_to_show. This functionality allow one to show the outline of the detected objects on the grayscale image used
    for detection, and also show the bgr image zoomed in on each detected object.
    detected_frame = numpy array of uint8 specifying the image used for detection
    detected_rect = a list containing zero or more lists, each specifying a rectangle that bounds a detected object in detected_frame
    frame_to_show = an alternate image used to display the image contained within each detected_rect. Input MUST be a numpy array in order to turn this feature on
    print_detected = boolean-like flag when truthy prints the x, y, w, and h values specifying the rectangle bounding each detected object
    rect_color = tuple with three values specifying the (b, g, r) color value for displaying the detected objects
    rect_thickness = integer specifying the thickness of the lines defining the rectangle bounding each detected object
    """
    for i, (x, y, w, h) in enumerate(detected_rect):
        if print_detected:
            print(f"Object {i} Location: x={x}, y={y}, w={w}, h={h}")
        detected_frame = cv.rectangle(detected_frame, (x, y), (x+w, y+h), rect_color, thickness=rect_thickness)
        if isinstance(frame_to_show, np.ndarray):
            show_frame(frame_to_show[y:y+h, x:x+w], "Objects Detected in Image")
    return detected_frame

### Load an Example Image and Convert to Grayscale

In [5]:
people = ["Megawati Sukarnoputri"] # for testing purposes
person = people[0]
img_num = "0017"
#img_num = "0027"
img_ext = ".jpg"

In [6]:
# Parse parameters to load a single grayscale image
per_fname = "_".join(person.split(" "))
img_path = os.path.join(dir_root, per_fname, "_".join([per_fname, img_num])+img_ext)
img_face = cv.imread(img_path)
gray_face = cv.cvtColor(img_face, cv.COLOR_BGR2GRAY)
print(f"Size of Example Grayscale Image = {gray_face.shape}")

Size of Example Grayscale Image = (250, 250)


## Face Detection

The first step is to download Haar Cascades xml file for frontal face detection from the following link:

https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml

If you want to follow the code below, then you should also download the eye and smile detection xml files from the haarcascades folder.

In [7]:
haar_cascade_params = {
    'face': {
        'xml_file': '/home/fdpearce/Documents/Projects/models/haar_cascades/haar_cascade_frontalface.xml',
        'params': {
            'scaleFactor': 1.1,
            'minNeighbors': 4,
           'minSize': (20, 20)
        }
    },
    'eye': {
        'xml_file': '/home/fdpearce/Documents/Projects/models/haar_cascades/haar_cascade_eye.xml',
        'params': {
            'scaleFactor': 1.1,
            'minNeighbors': 5,
           'minSize': (20, 20)
        }
    },
    'smile': {
        'xml_file': '/home/fdpearce/Documents/Projects/models/haar_cascades/haar_cascade_smile.xml',
        'params': {
            'scaleFactor': 1.1,
            'minNeighbors': 20,
           'minSize': (20, 20)
        }     
    }
}
haar_image = gray_face.copy()

In [8]:
# Face detection
detection_obj = 'face'
haar_obj = haar_cascade_params[detection_obj]
detected_face_rects = detect_primary_objects(haar_image, haar_obj['xml_file'], verbose=True, **haar_obj['params'])

Initial # of Objects Detected = 1
Final # of Objects Detected = 1


In [9]:
# Eye detection
detection_obj = 'eye'
haar_obj = haar_cascade_params[detection_obj]
detected_eye_rects = detect_primary_objects(haar_image, haar_obj['xml_file'], verbose=True, **haar_obj['params'])

Initial # of Objects Detected = 2
Iteration # = 1
minNeighbors = 7
Iteration # = 2
minNeighbors = 11
Iteration # = 3
minNeighbors = 17
Iteration # = 4
minNeighbors = 25
Iteration # = 5
minNeighbors = 35
Iteration # = 6
minNeighbors = 47
Iteration # = 7
minNeighbors = 61
Iteration # = 8
minNeighbors = 77
Iteration # = 9
minNeighbors = 76
Iteration # = 10
Maximum # of iterations (10) reached!
minNeighbors = 74
Final # of Objects Detected = 1


In [10]:
# Smile detection
detection_obj = 'smile'
haar_obj = haar_cascade_params[detection_obj]
detected_smile_rects = detect_primary_objects(haar_image, haar_obj['xml_file'], verbose=True, **haar_obj['params'])

Initial # of Objects Detected = 2
Iteration # = 1
minNeighbors = 22
Iteration # = 2
minNeighbors = 26
Iteration # = 3
minNeighbors = 32
Final # of Objects Detected = 1


In [11]:
print(f"# of Faces Found = {len(detected_face_rects)}")
print(f"# of Eyes Found = {len(detected_eye_rects)}")
print(f"# of Smiles Found = {len(detected_smile_rects)}")

# of Faces Found = 1
# of Eyes Found = 1
# of Smiles Found = 1


In [12]:
# Print detected smile(s) coordinates, and optionally show on original image
haar_image = show_detected_objects(haar_image, detected_smile_rects, print_detected=True, frame_to_show=None, rect_color=(255, 0, 0))

Object 0 Location: x=95, y=142, w=58, h=29


In [13]:
# Print detected eye(s) coordinates, and optionally show on original image
haar_image = show_detected_objects(haar_image, detected_eye_rects, print_detected=True, frame_to_show=None, rect_color=(0, 0, 0))

Object 0 Location: x=131, y=99, w=30, h=30


In [14]:
# Print detected face(s) coordinates, and optionally show on original image
haar_image = show_detected_objects(haar_image, detected_face_rects, print_detected=True, frame_to_show=None, rect_color=(0, 0, 0))

Object 0 Location: x=68, y=67, w=119, h=119


In [15]:
show_frame(haar_image, f"Faces(s) Detected in Image of {person}")

## Face Recognition

### Load Training Data

In [ ]:
person_img_max = 20
haar_face = "haar_cascade_frontalface.xml" # Path to haar cascade xml file
face_params = {'scaleFactor': 1.1,
               'minNeighbors': 4,
               'minSize': (20, 20)}
features, labels = create_training_data(dir_root, people, person_img_max, haar_face, face_params, detect_objects=False, verbose=True)

In [ ]:
print(f"# of Features = {len(features)}")
print(f"# of Labels = {len(labels)}")

In [ ]:
# Visually inspect example images and the bounding box detected by the face recognition algorithm
# Use the verbose setting above to print out bounding boxes
person = people[0]
img_num = ["0017", "0012", "0014", "0029", "0013", "0027", "0020"]
img_ext = ".jpg"
face_rectangles = [[[72, 69, 113, 113]], \
              [[73, 73, 110, 110]], \
              [[61, 61, 126, 126]], \
              [[64, 63, 123, 123]], \
              [[68, 65, 119, 119]], \
              [[70, 65, 118, 118]], \
              [[70, 69, 113, 113]]]

In [ ]:
for ind, im_num in enumerate(img_num):
    per_fname = "_".join(person.split(" "))
    img_path = os.path.join(dir_root, per_fname, "_".join([per_fname, im_num])+img_ext)
    img_face = cv.imread(img_path)
    gray_face = cv.cvtColor(img_face, cv.COLOR_BGR2GRAY)
    gray_face = show_detected_objects(gray_face, face_rectangles[ind], print_detected=True, frame_to_show=None, rect_color=(0, 0, 0))
    show_frame(gray_face, f"Primary Face Detected for {person}")

### 2. Train Face Recognition Model

In [ ]:
save_recognizer_data = False
face_recognizer = cv.face.LBPHFaceRecognizer_create()

In [ ]:
features = np.array(features, dtype="object")
labels = np.array(labels)

In [ ]:
face_recognizer.train(features, labels)

In [ ]:
if save_recognizer_data:
    face_recognizer.save("face_recognizer.yml")
    np.save("features.npy", features)
    np.save("labels.npy", labels)